# Load library

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearnex import patch_sklearn
patch_sklearn()
import warnings
warnings.filterwarnings("ignore")
import torch


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [3]:
import os 
os.system("pwd")

/root/LTIntruderDetection


0

# Load data

In [4]:
data = pd.read_csv('./dataset.csv')
data

,Class,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,NaN,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,NaN,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,NaN,NaN
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,NaN,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,NaN,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84952,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,NaN,no-cache,invalid,NaN,NaN,NaN
84953,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,NaN,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN
84954,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,NaN,invalid,invalid,NaN,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
84955,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,NaN,invalid,invalid,NaN,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


# EDA

## Data Understanding

In [5]:
data.shape

(84957, 13)

In [6]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84957 entries, 0 to 84956
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Class            84957 non-null  object
 1   Method           84957 non-null  object
 2   Host-Header      84957 non-null  object
 3   Connection       84953 non-null  object
 4   Accept           84916 non-null  object
 5   Accept-Charset   84867 non-null  object
 6   Accept-Language  84867 non-null  object
 7   Cache-control    84596 non-null  object
 8   Pragma           84210 non-null  object
 9   User-Agent       84954 non-null  object
 10  Content-Type     21950 non-null  object
 11  POST-Data        21942 non-null  object
 12  GET-Query        34707 non-null  object
dtypes: object(13)
memory usage: 8.4+ MB


In [7]:
data.describe()


,Class,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
count,84957,84957,84957,84953,84916,84867,84867,84596,84210,84954,21950,21942,34707
unique,2,3,2,3,6791,9053,13,3068,2,11677,1,16051,28395
top,Valid,GET,HTTP/1.1,close,"text/xml,application/xml,application/xhtml+xml...","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,no-cache,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,application/x-www-form-urlencoded,B2=Vaciar+carrito,errorMsg=Credenciales+incorrectas
freq,46287,63007,73067,69078,61065,61065,61065,63907,72544,61065,21950,1046,1000


In [8]:
print(f"Number of missing values: {data.isnull().sum().sum()}")
print(f"Number of duplicated values: {data.duplicated().sum()}")
print(f"Number of unique values: {data.nunique().sum()}")

Number of missing values: 177608
Number of duplicated values: 37114
Number of unique values: 75061


In [9]:
data["Class"].value_counts()

Class
Valid        46287
Anomalous    38670
Name: count, dtype: int64

## Data Cleaning

In [10]:
print(data["POST-Data"].isna().sum())

data = data.fillna("")
data

63015


,Class,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,,,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,,
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,,,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,,,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84952,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,,no-cache,invalid,,,
84953,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,
84954,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,,invalid,invalid,,,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
84955,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,,invalid,invalid,,,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


In [11]:
# Rename columns
data = data.rename(columns={"Class": "Label"})
data

,Label,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,,,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,,
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,,,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,,,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84952,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,,no-cache,invalid,,,
84953,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,
84954,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,,invalid,invalid,,,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
84955,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,,invalid,invalid,,,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


In [12]:
data = data.drop_duplicates()
print(f"Number of duplicated values: {data.duplicated().sum()}")


Number of duplicated values: 0


In [13]:
data


,Label,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,,,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,,
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,,,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,,,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84952,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,,no-cache,invalid,,,
84953,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,
84954,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,,invalid,invalid,,,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
84955,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,,invalid,invalid,,,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


In [14]:
# Outlier Detection
from scipy import stats
z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))
data = data[(z_scores < 3).all(axis=1)]
data


,Label,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,,,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,,
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,,,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,,,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84952,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,,no-cache,invalid,,,
84953,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,
84954,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,,invalid,invalid,,,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
84955,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,,invalid,invalid,,,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


## Descriptive Statistics


In [15]:
data.describe()


,Label,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
count,47843,47843,47843,47843,47843,47843,47843,47843,47843,47843,47843,47843,47843
unique,2,3,2,4,6792,9054,14,3069,3,11678,2,16052,28396
top,Anomalous,GET,HTTP/1.1,close,"text/xml,application/xml,application/xhtml+xml...","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,no-cache,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,,,
freq,27954,31763,35954,31965,23954,23954,23954,26794,35431,23954,31763,31771,18900


## Data Distribution

## Data Relationships



In [16]:

data.to_csv("data_no_outliers.csv", index=False)


### Features selection

In [17]:
data = pd.read_csv("data_no_outliers.csv")
data


,Label,Method,Host-Header,Connection,Accept,Accept-Charset,Accept-Language,Cache-control,Pragma,User-Agent,Content-Type,POST-Data,GET-Query
0,Anomalous,GET,HTTP/1.0,invalid,"audio/*;q=0.7, audio/*;q=0.0",*;q=0.6,*;q=0.3,invalid,NaN,Mozilla/9.9 (X11; U; Unix 0.4; el-1y; rv:6.5.0...,NaN,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...
1,Anomalous,PUT,HTTP/1.0,close,"image/*, image/jpeg;q=0.1","x-mac-greek;q=0.9, euc-cn, x-mac-greek, utf-7",*,invalid,no-cache,Mozilla/9.9 (X11; U; Open BSD i586 4.2; w1-ff;...,application/x-www-form-urlencoded,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN
2,Anomalous,POST,HTTP/1.0,close,"audio/*, audio/basic, text/html;q=0.4",*,non-standard,invalid,no-cache,Mozilla/9.9 (Windows; U; Windows NT 4.9; lf-wc...,application/x-www-form-urlencoded,NaN,NaN
3,Anomalous,GET,HTTP/1.0,close,*/*;q=0.3,"x-mac-japanese;q=0.2, iso-8859-3",*,invalid,no-cache,Mozilla/9.9 (Windows; U; Win 9x 9.4; 0t-os; rv...,NaN,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...
4,Anomalous,GET,HTTP/1.1,close,*/*,*,non-standard,invalid,no-cache,Mozilla/9.9 (Machintosh; U; PPC 8.5; in-hu; rv...,NaN,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47838,Valid,GET,HTTP/1.0,close,"application/*;q=0.5, text/*",windows-1255,non-standard,NaN,no-cache,invalid,NaN,NaN,NaN
47839,Valid,PUT,HTTP/1.1,close,*/*,"iso-2022-jp;q=0.9, koi8, shift_jis;q=0.8, iso-...",non-standard,NaN,no-cache,invalid,application/x-www-form-urlencoded,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN
47840,Valid,GET,HTTP/1.1,keep-alive,"image/*;q=0.3, image/*;q=0.4",x-mac-korean;q=0.7,non-standard,NaN,invalid,invalid,NaN,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...
47841,Valid,GET,HTTP/1.1,keep-alive,"video/*, image/*, application/x-tar","utf-8;q=0.3, utf-8;q=0.4",non-standard,NaN,invalid,invalid,NaN,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr


In [18]:
newData = data[["POST-Data", "GET-Query","Host-Header","Method","Content-Type","Label"]]
newData

,POST-Data,GET-Query,Host-Header,Method,Content-Type,Label
0,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...,HTTP/1.0,GET,NaN,Anomalous
1,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN,HTTP/1.0,PUT,application/x-www-form-urlencoded,Anomalous
2,NaN,NaN,HTTP/1.0,POST,application/x-www-form-urlencoded,Anomalous
3,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...,HTTP/1.0,GET,NaN,Anomalous
4,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...,HTTP/1.1,GET,NaN,Anomalous
...,...,...,...,...,...,...
47838,NaN,NaN,HTTP/1.0,GET,NaN,Valid
47839,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN,HTTP/1.1,PUT,application/x-www-form-urlencoded,Valid
47840,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...,HTTP/1.1,GET,NaN,Valid
47841,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr,HTTP/1.1,GET,NaN,Valid


In [19]:
# Feature Engineering
from sklearn.preprocessing import LabelEncoder

# Encode the label
label_encoder = LabelEncoder()
newData["Label"] = label_encoder.fit_transform(newData["Label"])
newData


,POST-Data,GET-Query,Host-Header,Method,Content-Type,Label
0,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...,HTTP/1.0,GET,NaN,0
1,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN,HTTP/1.0,PUT,application/x-www-form-urlencoded,0
2,NaN,NaN,HTTP/1.0,POST,application/x-www-form-urlencoded,0
3,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...,HTTP/1.0,GET,NaN,0
4,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...,HTTP/1.1,GET,NaN,0
...,...,...,...,...,...,...
47838,NaN,NaN,HTTP/1.0,GET,NaN,1
47839,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN,HTTP/1.1,PUT,application/x-www-form-urlencoded,1
47840,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...,HTTP/1.1,GET,NaN,1
47841,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr,HTTP/1.1,GET,NaN,1


In [20]:
# Encode the categorical features
newData["POST-Data"] = label_encoder.fit_transform(newData["POST-Data"].astype(str))
newData["GET-Query"] = label_encoder.fit_transform(newData["GET-Query"].astype(str))
newData["Host-Header"] = label_encoder.fit_transform(newData["Host-Header"].astype(str))
newData["Method"] = label_encoder.fit_transform(newData["Method"].astype(str))
newData["Content-Type"] = label_encoder.fit_transform(newData["Content-Type"].astype(str))
newData



,POST-Data,GET-Query,Host-Header,Method,Content-Type,Label
0,14731,3145,0,0,1,0
1,15585,22717,0,2,0,0
2,14731,22717,0,1,0,0
3,14731,11732,0,0,1,0
4,14731,5443,1,0,1,0
...,...,...,...,...,...,...
47838,14731,22717,0,0,1,1
47839,4936,22717,1,2,0,1
47840,14731,28370,1,0,1,1
47841,14731,12278,1,0,1,1


In [21]:
# Ensure k is less than or equal to the number of features
from sklearn.feature_selection import SelectKBest
k = min(5, newData.shape[1] - 1)  # Subtract 1 for the label column

# Select the top k features
selector = SelectKBest(k=k)
selector.fit(newData.drop("Label", axis=1), newData["Label"])

# Debugging: Print shapes to verify
print("DataFrame shape:", newData.drop("Label", axis=1).shape)
print("Support array shape:", selector.get_support().shape)

# Get the selected feature names
selected_features = newData.drop("Label", axis=1).columns[selector.get_support()].tolist()
selected_features



DataFrame shape: (47843, 5)
Support array shape: (5,)


['POST-Data', 'GET-Query', 'Host-Header', 'Method', 'Content-Type']

In [22]:
data = data[selected_features + ["Label"]]  
data


,POST-Data,GET-Query,Host-Header,Method,Content-Type,Label
0,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...,HTTP/1.0,GET,NaN,Anomalous
1,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN,HTTP/1.0,PUT,application/x-www-form-urlencoded,Anomalous
2,NaN,NaN,HTTP/1.0,POST,application/x-www-form-urlencoded,Anomalous
3,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...,HTTP/1.0,GET,NaN,Anomalous
4,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...,HTTP/1.1,GET,NaN,Anomalous
...,...,...,...,...,...,...
47838,NaN,NaN,HTTP/1.0,GET,NaN,Valid
47839,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN,HTTP/1.1,PUT,application/x-www-form-urlencoded,Valid
47840,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...,HTTP/1.1,GET,NaN,Valid
47841,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr,HTTP/1.1,GET,NaN,Valid


In [23]:
data['Method_httpversion'] = data['Method'] + " " + data['Host-Header']
data

,POST-Data,GET-Query,Host-Header,Method,Content-Type,Label,Method_httpversion
0,NaN,NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs...,HTTP/1.0,GET,NaN,Anomalous,GET HTTP/1.0
1,tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin...,NaN,HTTP/1.0,PUT,application/x-www-form-urlencoded,Anomalous,PUT HTTP/1.0
2,NaN,NaN,HTTP/1.0,POST,application/x-www-form-urlencoded,Anomalous,POST HTTP/1.0
3,NaN,imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8...,HTTP/1.0,GET,NaN,Anomalous,GET HTTP/1.0
4,NaN,beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3D...,HTTP/1.1,GET,NaN,Anomalous,GET HTTP/1.1
...,...,...,...,...,...,...,...
47838,NaN,NaN,HTTP/1.0,GET,NaN,Valid,GET HTTP/1.0
47839,lYVd1L2eincludenK=56&EbvGT=9078475&edftegaG6Ow...,NaN,HTTP/1.1,PUT,application/x-www-form-urlencoded,Valid,PUT HTTP/1.1
47840,NaN,zsnd0edcIssro=00445431&o2d=bStgeseuew0U%5Dm%40...,HTTP/1.1,GET,NaN,Valid,GET HTTP/1.1
47841,NaN,kesRt=leodn%25&daeAzrero=5&shavingI0=sr,HTTP/1.1,GET,NaN,Valid,GET HTTP/1.1


In [24]:
def concatenate_features(row):
    return f"{row['Method_httpversion']} | {row['GET-Query']} | {row['Content-Type']} | {row['POST-Data']}"

data['combined_text'] = data.apply(concatenate_features, axis=1)
data = data.drop(columns=['Method'])
data = data.drop(columns=['Host-Header'])

data = data[['combined_text','Label']]
data 


,combined_text,Label
0,GET HTTP/1.0 | NT0cDJacceptnfOF=138695082&wcat...,Anomalous
1,PUT HTTP/1.0 | nan | application/x-www-form-ur...,Anomalous
2,POST HTTP/1.0 | nan | application/x-www-form-u...,Anomalous
3,GET HTTP/1.0 | imgbo-K@xZdW=tnpp&QVc@=8Ehto&Ea...,Anomalous
4,GET HTTP/1.1 | beqqeAoaUeLxen=%22%29%28targetf...,Anomalous
...,...,...
47838,GET HTTP/1.0 | nan | nan | nan,Valid
47839,PUT HTTP/1.1 | nan | application/x-www-form-ur...,Valid
47840,GET HTTP/1.1 | zsnd0edcIssro=00445431&o2d=bStg...,Valid
47841,GET HTTP/1.1 | kesRt=leodn%25&daeAzrero=5&shav...,Valid


In [25]:
from sklearn.preprocessing import LabelEncoder
# Labelencode the label
le = LabelEncoder()
data["Label"] = le.fit_transform(data["Label"])
data

,combined_text,Label
0,GET HTTP/1.0 | NT0cDJacceptnfOF=138695082&wcat...,0
1,PUT HTTP/1.0 | nan | application/x-www-form-ur...,0
2,POST HTTP/1.0 | nan | application/x-www-form-u...,0
3,GET HTTP/1.0 | imgbo-K@xZdW=tnpp&QVc@=8Ehto&Ea...,0
4,GET HTTP/1.1 | beqqeAoaUeLxen=%22%29%28targetf...,0
...,...,...
47838,GET HTTP/1.0 | nan | nan | nan,1
47839,PUT HTTP/1.1 | nan | application/x-www-form-ur...,1
47840,GET HTTP/1.1 | zsnd0edcIssro=00445431&o2d=bStg...,1
47841,GET HTTP/1.1 | kesRt=leodn%25&daeAzrero=5&shav...,1


In [26]:
def preprocess_request(request):
    parts = request.split(' | ')
    method_httpversion,query_param,content_type,body  = (parts + ["UNKNOWN"] * 4)[:4]
    return (
        f"HTTP: {method_httpversion}, "
        f"Query: {query_param}, "
        f"Content-Type: {content_type}, "
        f"Body: {body}"
    )

data['Final'] = data['combined_text'].apply(preprocess_request)
data

,combined_text,Label,Final
0,GET HTTP/1.0 | NT0cDJacceptnfOF=138695082&wcat...,0,"HTTP: GET HTTP/1.0, Query: NT0cDJacceptnfOF=13..."
1,PUT HTTP/1.0 | nan | application/x-www-form-ur...,0,"HTTP: PUT HTTP/1.0, Query: nan, Content-Type: ..."
2,POST HTTP/1.0 | nan | application/x-www-form-u...,0,"HTTP: POST HTTP/1.0, Query: nan, Content-Type:..."
3,GET HTTP/1.0 | imgbo-K@xZdW=tnpp&QVc@=8Ehto&Ea...,0,"HTTP: GET HTTP/1.0, Query: imgbo-K@xZdW=tnpp&Q..."
4,GET HTTP/1.1 | beqqeAoaUeLxen=%22%29%28targetf...,0,"HTTP: GET HTTP/1.1, Query: beqqeAoaUeLxen=%22%..."
...,...,...,...
47838,GET HTTP/1.0 | nan | nan | nan,1,"HTTP: GET HTTP/1.0, Query: nan, Content-Type: ..."
47839,PUT HTTP/1.1 | nan | application/x-www-form-ur...,1,"HTTP: PUT HTTP/1.1, Query: nan, Content-Type: ..."
47840,GET HTTP/1.1 | zsnd0edcIssro=00445431&o2d=bStg...,1,"HTTP: GET HTTP/1.1, Query: zsnd0edcIssro=00445..."
47841,GET HTTP/1.1 | kesRt=leodn%25&daeAzrero=5&shav...,1,"HTTP: GET HTTP/1.1, Query: kesRt=leodn%25&daeA..."


In [27]:
# remove combine_text
data = data.drop(columns="combined_text")
data

,Label,Final
0,0,"HTTP: GET HTTP/1.0, Query: NT0cDJacceptnfOF=13..."
1,0,"HTTP: PUT HTTP/1.0, Query: nan, Content-Type: ..."
2,0,"HTTP: POST HTTP/1.0, Query: nan, Content-Type:..."
3,0,"HTTP: GET HTTP/1.0, Query: imgbo-K@xZdW=tnpp&Q..."
4,0,"HTTP: GET HTTP/1.1, Query: beqqeAoaUeLxen=%22%..."
...,...,...
47838,1,"HTTP: GET HTTP/1.0, Query: nan, Content-Type: ..."
47839,1,"HTTP: PUT HTTP/1.1, Query: nan, Content-Type: ..."
47840,1,"HTTP: GET HTTP/1.1, Query: zsnd0edcIssro=00445..."
47841,1,"HTTP: GET HTTP/1.1, Query: kesRt=leodn%25&daeA..."


In [28]:
data["Final"][1]

"HTTP: PUT HTTP/1.0, Query: nan, Content-Type: application/x-www-form-urlencoded, Body: tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin gcxml&opi49=ornIhtaccespca&pzu=t8yb']   "

# Data preprocessing

## Tokenize

In [29]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [30]:
# find the max length of the data
max_length = max(map(len, data['Final']))
max_length


924

In [31]:
dataset = [str(i) for i in data['Final'].values]
dataset

['HTTP: GET HTTP/1.0, Query: NT0cDJacceptnfOF=138695082&wcatUzz=dmH_g&t1lTs=cJXD0&Ees=ehdoi0ma21shumta&ird4ksfotUq=tWF.ZQ&0zsautyt=imss3iw&utn=Yo4a%2Bj%7Cy+teefl&igiebeetsefet=%29+%28+++%7C++%28displayName%3Dhad*%29++%28name++++%3D+++had*+++%29%28mail%3Dhad*+++%29&ru=rs3O0-hpD&lHle=sC6&eanede=et3Feoptavotgaeen%3Evoc&oyareNneEolwot=n+raD&AImidura=o%3FeRe%40yt+5onX&Ax1ao=3490, Content-Type: nan, Body: nan',
 "HTTP: PUT HTTP/1.0, Query: nan, Content-Type: application/x-www-form-urlencoded, Body: tcy0wLnt7sw=lTWFH@dvrU&zUEX8MFk=htaccesonDonin gcxml&opi49=ornIhtaccespca&pzu=t8yb']   ",
 'HTTP: POST HTTP/1.0, Query: nan, Content-Type: application/x-www-form-urlencoded, Body: nan',
 'HTTP: GET HTTP/1.0, Query: imgbo-K@xZdW=tnpp&QVc@=8Ehto&EaEEtEgdis=thxoo8n&aaaeo1sslic=ra&6tbaI6ca=..%2F..%2F..%2Fusr%2Fdsqqdsqsd.xml&V60Nr4D6=n6H&qj8y3=1989&Rca=fng&1niaceenxleA=tui, Content-Type: nan, Body: nan',
 'HTTP: GET HTTP/1.1, Query: beqqeAoaUeLxen=%22%29%28targetfilter%3D%28o%3DNetscapeRoot%29%29&wrAnt

In [32]:
tokenized_data = tokenizer(dataset,padding=True, truncation=True, max_length=max_length,return_tensors="pt").to(device)
tokenized_data

{'input_ids': tensor([[ 101, 8299, 1024,  ...,    0,    0,    0],
        [ 101, 8299, 1024,  ...,    0,    0,    0],
        [ 101, 8299, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 8299, 1024,  ...,    0,    0,    0],
        [ 101, 8299, 1024,  ...,    0,    0,    0],
        [ 101, 8299, 1024,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

# Modeling

In [59]:
import torch
from transformers import DistilBertForSequenceClassification
from flash_attn.modules.mha import FlashSelfAttention

id2label = {0: "Attack", 1: "Benign"}
label2id = {"Attack": 0, "Benign": 1}
model = DistilBertForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id,torch_dtype=torch.float16, attn_implementation="flash_attention_2")
for layer in model.distilbert.transformer.layer:
    layer.attention = FlashSelfAttention()
model = model.to(device)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): FlashSelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=768, out_features=3072, bias=True)
            (lin2): Linear(in_features=3072, out_features=768, bias=True)
            (activation): GELUActivation()
          )
          (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        )
      )


In [60]:
tokenized_data.keys()

dict_keys(['input_ids', 'attention_mask'])

In [61]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    lr: 5e-05
    weight_decay: 0.0
)

In [69]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW
from tqdm import tqdm
import plotly.graph_objects as go

# Assuming you have data and tokenized_data prepared
labels = torch.tensor(data["Label"].values)  # Target column
dataset = TensorDataset(tokenized_data["input_ids"], tokenized_data["attention_mask"], labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training parameters
epochs = 10
patience = 3
min_val_loss = float("inf")
early_stop_counter = 0

# Metrics storage
training_loss_per_epoch = []
validation_loss_per_epoch = []
training_accuracy_per_epoch = []
validation_accuracy_per_epoch = []

# Training and validation loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")
    train_loss = 0.0
    val_loss = 0.0
    train_correct = 0
    train_total = 0
    val_correct = 0
    val_total = 0

    # Training phase
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}")
    for batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        predictions = torch.argmax(logits, dim=1)
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
        train_loss += loss.item()

        progress_bar.set_postfix({"Train Loss": loss.item()})

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = train_correct / train_total
    training_loss_per_epoch.append(avg_train_loss)
    training_accuracy_per_epoch.append(train_accuracy)

    # Validation phase
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            # Calculate validation accuracy
            predictions = torch.argmax(logits, dim=1)
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total
    validation_loss_per_epoch.append(avg_val_loss)
    validation_accuracy_per_epoch.append(val_accuracy)

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        early_stop_counter += 1
        print(f"No improvement in validation loss. Early stopping counter: {early_stop_counter}/{patience}")
        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 of 10


Training Epoch 1: 100%|███████████████████████████████████████████| 2393/2393 [06:14<00:00,  6.39it/s, Train Loss=0.204]


Validation Loss: 0.2036, Validation Accuracy: 0.9142
Epoch 2 of 10


Training Epoch 2: 100%|████████████████████████████████████████| 2393/2393 [06:15<00:00,  6.38it/s, Train Loss=0.000132]


...

Validation Loss: 0.1917, Validation Accuracy: 0.9196
No improvement in validation loss. Early stopping counter: 1/3
Epoch 4 of 10


Training Epoch 4: 100%|███████████████████████████████████████████| 2393/2393 [06:15<00:00,  6.37it/s, Train Loss=0.177]


Validation Loss: 0.1789, Validation Accuracy: 0.9250
No improvement in validation loss. Early stopping counter: 2/3
Epoch 5 of 10


Training Epoch 5: 100%|█████████████████████████████████████████| 2393/2393 [06:14<00:00,  6.39it/s, Train Loss=0.00272]


Validation Loss: 0.1831, Validation Accuracy: 0.9263
No improvement in validation loss. Early stopping counter: 3/3
Early stopping triggered!


In [70]:
# Plot training and validation loss and accuracy
fig = go.Figure()

# Loss plots
fig.add_trace(go.Scatter(
    x=list(range(1, len(training_loss_per_epoch) + 1)),
    y=training_loss_per_epoch,
    mode='lines+markers',
    name='Training Loss'
))
fig.add_trace(go.Scatter(
    x=list(range(1, len(validation_loss_per_epoch) + 1)),
    y=validation_loss_per_epoch,
    mode='lines+markers',
    name='Validation Loss'
))

# Accuracy plots
fig.add_trace(go.Scatter(
    x=list(range(1, len(training_accuracy_per_epoch) + 1)),
    y=training_accuracy_per_epoch,
    mode='lines+markers',
    name='Training Accuracy'
))
fig.add_trace(go.Scatter(
    x=list(range(1, len(validation_accuracy_per_epoch) + 1)),
    y=validation_accuracy_per_epoch,
    mode='lines+markers',
    name='Validation Accuracy'
))

fig.update_layout(
    title="Training and Validation Metrics",
    xaxis_title="Epoch",
    yaxis_title="Metrics",
    template="plotly_white",
    legend=dict(title="Metrics")
)


In [72]:
# Save the model
torch.save(model.state_dict(), "model.pth")
print(os.system("ls"))

best_model.pt
data_cleaned.csv
data_no_outliers.csv
dataset.csv
index.ipynb
model.pth
0
